In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

citations:
https://www.geeksforgeeks.org/data-analysis/exploratory-data-analysis-in-python/


In [2]:
TWEETS_PATH= "../data/raw_tweets_text.csv"
SENTIMENT_PATH="../data/t4sa_text_sentiment.tsv"


load + small transformations for more readable dataset

In [3]:
#load data
tweets_df = pd.read_csv(TWEETS_PATH, encoding='latin-1', header=0)
sentiment_df= pd.read_csv(SENTIMENT_PATH, sep='\t', header=0)


## Get Basic information

In [4]:
print(tweets_df.shape)
print(sentiment_df.shape)

(3452663, 2)
(1179957, 4)


In [5]:
print(tweets_df.info())
print(sentiment_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3452663 entries, 0 to 3452662
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   text    object
dtypes: int64(1), object(1)
memory usage: 52.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179957 entries, 0 to 1179956
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   TWID    1179957 non-null  int64  
 1   NEG     1179957 non-null  float64
 2   NEU     1179957 non-null  float64
 3   POS     1179957 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 36.0 MB
None


In [ ]:

# split second column of tweets into first word, main tweet, link 
# Process in chunks to avoid running out of memory
chunk_size = 10000
num_rows = tweets_df.shape[0]

main_text = []
last_link = []

for start in range(0, num_rows, chunk_size):
    end = min(start + chunk_size, num_rows)
    text_chunk = tweets_df['text'].iloc[start:end].str.split()
    last_link.extend(text_chunk.str[-1].where(text_chunk.str[-1].str.startswith('http'), ''))
    main_text.extend(text_chunk.apply(lambda x: ' '.join(x[:-1]) if x and x[-1].startswith('http') else ' '.join(x) if x else ''))

tweets_df['main_text'] = main_text
tweets_df['last_link'] = last_link



In [ ]:
tweets_df.nunique()

first_word
RT       667323
I         63164
Found     62819
The       53542
Come      42622
My        25809
This      25261
Happy     23041
New       22490
How       19895
Name: count, dtype: int64


In [ ]:
matching_ids = set(tweets_df['id']).intersection(set(sentiment_df['TWID']))

exclusive_tweets_df = tweets_df[~tweets_df['id'].isin(matching_ids)]
exclusive_sentiment_df = sentiment_df[~sentiment_df['TWID'].isin(matching_ids)]

merged_df = pd.merge(
    exclusive_tweets_df,
    exclusive_sentiment_df,
    left_on='id',
    right_on='TWID',
    how='outer'
)



Number of matching IDs: 1179957


# EDA section

do your eda here
you probably want to create a copy of the dataset before making any transformations(so you dont have to reload the entire notebook if you make a mistake)